In [1]:
# Import necessary libraries
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [2]:
# Load each CSV file into the respective dataframe from August,2020 to July,2021
# Load original .csv files, a year's worth of data from August 2020 to July 2021
aug08_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202008-divvy-tripdata.csv")
sep09_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202009-divvy-tripdata.csv")
oct10_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202010-divvy-tripdata.csv")
nov11_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202011-divvy-tripdata.csv")
dec12_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202012-divvy-tripdata.csv")
jan01_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202101-divvy-tripdata.csv")
feb02_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202102-divvy-tripdata.csv")
mar03_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202103-divvy-tripdata.csv")
apr04_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202104-divvy-tripdata.csv")
may05_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202105-divvy-tripdata.csv")
jun06_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202106-divvy-tripdata.csv")
jul07_df = pd.read_csv("/content/drive/MyDrive/Google Data Analytics Capstone Project/202107-divvy-tripdata.csv")

In [3]:
# Merge all of the data frames into one year view
cyclistic_df = pd.concat([aug08_df, sep09_df, oct10_df, nov11_df, dec12_df, jan01_df, feb02_df, mar03_df, apr04_df, may05_df, jun06_df, jul07_df])

# Remove individual month data frames to clear up space in memory
del aug08_df, sep09_df, oct10_df, nov11_df, dec12_df, jan01_df, feb02_df, mar03_df, apr04_df, may05_df, jun06_df, jul07_df

# Create a new data frame to contain new columns
cyclistic_date = cyclistic_df.copy()

In [4]:
# Calculate ride length by subtracting ended_at time from started_at time and convert it to minutes
cyclistic_date['started_at'] = pd.to_datetime(cyclistic_date['started_at'])
cyclistic_date['ended_at'] = pd.to_datetime(cyclistic_date['ended_at'])
cyclistic_date['ride_length'] = (cyclistic_date['ended_at'] - cyclistic_date['started_at']).dt.total_seconds() / 60
cyclistic_date['ride_length'] = cyclistic_date['ride_length'].round(1)

In [14]:
# Create columns for: total_ride, day of week, month, day, year, time, hour
cyclistic_date['total_ride'] = cyclistic_date['ride_id'].value_counts().sum()/ 1_000_000
cyclistic_date['total_ride'] = cyclistic_date['total_ride'].apply(lambda x: f'{x:.1f} M')
cyclistic_date['date'] = cyclistic_date['started_at'].dt.date
cyclistic_date['day_of_week'] = cyclistic_date['started_at'].dt.strftime('%A')
cyclistic_date['month'] = cyclistic_date['started_at'].dt.strftime('%m')
cyclistic_date['day'] = cyclistic_date['started_at'].dt.strftime('%d')
cyclistic_date['year'] = cyclistic_date['started_at'].dt.strftime('%Y')
cyclistic_date['time'] = cyclistic_date['started_at'].dt.strftime('%H:%M:%S')
cyclistic_date['hour'] = cyclistic_date['started_at'].dt.hour

In [18]:
# Create columns for different seasons: Spring, Summer, Fall, Winter
cyclistic_date['season'] = cyclistic_date['month'].map({
    '03': 'Spring', '04': 'Spring', '05': 'Spring',
    '06': 'Summer', '07': 'Summer', '08': 'Summer',
    '09': 'Fall', '10': 'Fall', '11': 'Fall',
    '12': 'Winter', '01': 'Winter', '02': 'Winter'
})

In [19]:
# Create columns for different time_of_day: Night, Morning, Afternoon, Evening
cyclistic_date['time_of_day'] = cyclistic_date['hour'].apply(lambda hour: 'Night' if 0 <= hour < 6 else
                                                             'Morning' if 6 <= hour < 12 else
                                                             'Afternoon' if 12 <= hour < 18 else
                                                             'Evening')

In [20]:
# Create a column for the month using the full month name
cyclistic_date['month'] = cyclistic_date['month'].map({
    '01': 'January', '02': 'February', '03': 'March',
    '04': 'April', '05': 'May', '06': 'June',
    '07': 'July', '08': 'August', '09': 'September',
    '10': 'October', '11': 'November', '12': 'December'
})

In [ ]:
# Clean the data
cyclistic_date.dropna(inplace=True)  # Remove rows with NA values
cyclistic_date.drop_duplicates(inplace=True)  # Remove duplicate rows
cyclistic_date = cyclistic_date[cyclistic_date['ride_length'] > 0]  # Remove rows where ride_length is <= 0
cyclistic_date.drop(['ride_id', 'start_station_id', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng'], axis=1, inplace=True)

In [26]:
# View the final data
cyclistic_date.to_csv("cyclistic_data.csv", index=False)  # Save the data to a CSV file